# The Newsvendor Model - Section 4: Generalization

In Section 3 we saw that the analysis we did for the (discrete) $\text{Uniform}\{0,\dots,d\}$ distribution can be adapted to the (continuous) $\text{Exponential}(\lambda)$ distribution.  If you were able to do Exercise 3.4, you saw that this further generalized to any non-negative, continuous demand distribution.  The optimal solution satisfies the “critical fractile formula”:

$$ y^* = F^{-1}\left(\frac{p - c}{p}\right) $$

In this section, we will consider a more general model that can be adapted for variations of the basic Newsvendor.  In doing so, we will also provide some economic insight into the critical fractile formula and answer the question initially posed in Exercise 2.4: how can you think of the optimal order quantity in terms of balancing a tradeoff?

## Rephrasing the Original Problem

Recall that we had the following formula for profit:

$$ \text{profit} = \underbrace{p \min\{y, D\}}_{\text{revenue}} - \underbrace{c y}_{\text{cost}} $$

We're going to rewrite this slightly and introduce two new quantities: surplus and shortage.

$$ \text{profit} = (p - c) D - (p - c) \max\{D - y, 0\} - c \max\{y - D, 0\} $$

**Remark:** How do we know these two equations equal?  You can verify it with a bunch of algebra or look at the graphs of the function.  See the part “Sales, Shortage, Surplus (Optional)” below.  You may want to review the definition of the $\max$ function from the “Formulas” portion of Section 2.

Let's interpret the new expressions (recall that $D$ is the demand and $c$ is the wholesale cost):

* $(p - c)$ is the profit *per newspaper sold*.

* $\max\{D - y, 0\}$ is the “shortage” or the amount that demand $D$ exceeded our order quantity $y$.  This cannot be negative and decreases as $y$ increases.
    
* $\max\{y - D, 0\}$ is the “surplus” or the number of newspapers unsold newspapers at the end of the day.  This cannot be negative and increases as $y$ increases.

So we are writing the profit as:

* ideal total profit $(p - c) D$: the total profit if we were clairvoyant and exactly meet demand.
* total shortage cost $(p - c) \max\{D - y, 0\}$: the profit missed when we under-ordering (compared to the ideal)
* total surplus cost $c \max\{y - D, 0\}$: the unnecessary cost of unsold newspapers when we over-order (compared to the ideal)

Putting this together and using the shorthand notation $(D - y)^+ \overset{\mathsf{def}}{=} \max\{D - y, 0\}$ and $(y - D)^+ \overset{\mathsf{def}}{=} \max\{y - D, 0\}$ we have:

$$ \text{profit} = \underbrace{(p - c) D}_{\text{ideal total profit}} - \underbrace{(p - c) (D - y)^+}_{\text{total shortage cost}} - \underbrace{c (y - D)^+}_{\text{total surplus cost}} $$

So, in order to maximize expected profit, we want to find the order quantity $y$ that maximizes

$$ \mathsf{E}[\text{profit}] = (p - c) \mathsf{E}[D] - (p - c)\mathsf{E}\left[(D - y)^+\right] - c \mathsf{E}\left[(y - D)^+\right] $$

How have we been finding the optimal order quantity $y^*$?  The hint in Exercise 2.4 (which also worked in Exercises 3.2, 3.3, and 3.4) was to solve for where the derivative equals $0$.  In other words, the optimal order quantity $y^*$ has to satisfy

$$ \frac{d}{dy}\left(\mathsf{E}[\text{profit}]\right) = 0. $$

Using our rephrased profit function

$$ \underbrace{-\frac{d}{dy} \left((p - c) \mathsf{E}\left[(D - y)^+\right]\right)}_{\text{marginal cost of shortage}} = \underbrace{\frac{d}{dy} \left(c \mathsf{E}\left[(y - D)^+\right]\right)}_{\text{marginal cost of surplus}}. $$

In words, the optimal order quantity represents the point at which the benefit of ordering more inventory (since it reduces the shortage cost) begins to be outweighted by the cost of that inventory (since it increases the surplus cost).  Moreover, this is very compactly represented (and easily calculated) as:

$$ y^* = F^{-1}\left(\frac{\text{shortage cost}}{\text{shortage + surplus costs}}\right) $$

## General Formulation

The more general formulation of the newsvendor considers the problem in terms of the parameters:

* $a$ = per unit profit
* $b$ = shortage cost (per unit)
* $c$ = surplus cost (per unit)

and

$$ \text{profit} = a D - b (D - y)^+ - c (y - D)^+ $$

$$ \mathsf{E}[\text{profit}] = a \mathsf{E}[D] - b \mathsf{E}\left[(D - y)^+\right] - c \mathsf{E}\left[(y - D)^+\right] $$

with optimal order quantity given by the critical fractile formula

$$y^* = F^{-1}\left(\frac{b}{b+c}\right)$$

We can recover the original problem with $a = b = \text{retail price} - \text{wholesale cost}$ and $c = \text{wholesale cost}$.

In Homework 1, you will derive yet another the critical fractile formula and apply it to a setting with a feature that does not fit into the original formulation.

**Remark:** You may choose to use any problem formulation that can be applied to the problem at hand and is intuitive for you to understand.  For example, you may find that having a term for $\text{sales} = \min\{y, D\}$ is helpful -- you can build an expression for $\text{profit}$ that includes $\min\{y, D\}$, $(D - y)^+$, $(y - D)^+$, $D$, and $y$.  Just be sure to formulate the coefficients correctly.

## Visualization (Optional)

The following code repeats some of the analysis you did in Sections 3 but using the general formulation with $\text{shortage}$ and $\text{surplus}$ (and their associated costs) instead of $\text{sales}$ (and revenue).  If you use the same problem parameters, you should get the same answers and plots for $\text{profit}$ and $\mathsf{E}[\text{profit}]$ (subject to numerical accuracy or simulation error).  For a direct comparison, copy and paste the relevant codeblocks from Section 3.

In [1]:
# Import Python packages
import newsvendor
import news_plot

Loading BokehJS ...

In [2]:
# Initialize model
# Default exponential distribution has parameter lambda = 0.1
# Default model parameters are wholesale_cost = 1, retail_price = 2.
news = newsvendor.Newsvendor('exponential')

# Simulate demand
news.sim_data(num_days=365)

# Histograms with order quantity slider
bins = int(max(news.data['demand'])-min(news.data['demand'])+1)
hist_fields = ['demand', 'sales', 'profit']
hist_labels = {field: {'x': field, 'y': 'count', 'title': field + ' histogram'} for field in hist_fields}
hist_func = lambda n: news_plot.hist_glyphs(news.data, bins=bins)
newsvendor.interact_gridplot(news_plot.figures(hist_fields), hist_func, news, labels=hist_labels)

# Scatter plots with order quantity slider
scatter_fields = ['sales', 'revenue', 'profit']
scatter_labels = {field: {'x': 'demand', 'y': field, 'title': field} for field in scatter_fields}
scatter_func = lambda n: news_plot.scatter_glyphs(n.data, 'demand')
newsvendor.interact_gridplot(news_plot.figures(scatter_fields), scatter_func, news, labels=scatter_labels)

# Table of expected values with order quantity slider
ev_table_fields = ['demand', 'sales', 'profit']
newsvendor.order_qty_interact(lambda y: news.exp_vals([y])[ev_table_fields].style.hide_index(), news)

# Line plots of expected values as functions of order quantity
ev_fields = ['sales', 'revenue', 'cost', 'profit']
ev_labels = {field: {'x': 'order qty', 'y': 'expected ' + field, 'title': field} for field in ev_fields}
ev_func = lambda n: news_plot.line_glyphs(n.exp_vals(), 'order qty')
ev_sources, ev_plots = newsvendor.news_gridplot(news_plot.figures(ev_fields, plot_width=350, plot_height=350), ev_func, news, labels=ev_labels)
circle_func = lambda n: news_plot.circle_markers(n.exp_vals([n.order_qty]))
newsvendor.interact_gridplot({field: ev_plots[field] for field in ev_fields}, circle_func, news, ncols=2)

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…

## Sales, Shortage, Surplus (Optional)

It may be helpful to visualize the relationship between

* $\text{sales} = \min\{y, D\}$
* $\text{shortage} = (D - y)^+$
* $\text{surplus} = (y - D)^+$

We will consider these as functions of demand $D$.  You can verify graphically the following [identities](https://en.wikipedia.org/wiki/Identity_(mathematics)):

$$\begin{aligned}
    D - y &= (D - y)^+ - (y - D)^+ \\
    \min\{y, D\} &= D - (D - y)^+ = y - (y - D)^+
  \end{aligned}$$
  
With these two identities, you can show that the original formula for $\text{profit}$ and the rephrased formula are equivalent.

Conceptually, you may find it easier to think of these identities as:

$$\begin{aligned}
    \text{order quantity} &= \text{sales} + \text{surplus} \\
    \text{demand} &= \text{sales} + \text{shortage}
  \end{aligned}$$

Less relevant for the Newsvendor, but for practice, you can check that $|D - y| = (D - y)^+ + (y - D)^+$ (where $|\cdot|$ denotes absolute value).

In [3]:
import numpy as np
import pandas as pd

# Relationship between functions for demand, order quantity, sales, shortage, and surplus
def funcs(y):
    D_vals = pd.Index(np.arange(0, 30, 1), name='D')
    f = pd.DataFrame(index=D_vals)
    f['D'] = pd.Series(D_vals)
    f['y'] = pd.Series(y, index=D_vals)
    f['min{y, D}'] = np.minimum(y, f['D'])
    f['(D - y)^+'] = np.maximum(f['D'] - y, 0)
    f['(y - D)^+'] = np.maximum(y - f['D'], 0)
    f['D - y'] = f['D'] - y
    
    return f

fields = funcs(news.order_qty).columns.values
labels = {field: {'x': 'D', 'y': field, 'title': field} for field in fields}
line_plot_func = lambda n: news_plot.line_glyphs(funcs(n.order_qty), 'D')
newsvendor.interact_gridplot(news_plot.figures(fields), line_plot_func, news, labels=labels)

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…